# Video Translator con Clonazione Vocale

Questo notebook dimostra come utilizzare il progetto Video Translator per tradurre video con clonazione vocale e sincronizzazione avanzata.

**Versione:** 0.5.1

## Caratteristiche
- Trascrizione automatica del parlato utilizzando Whisper
- Traduzione automatica del testo trascritto utilizzando MBart
- Clonazione vocale e sintesi vocale con Tortoise-TTS
- Separazione di voci e musica di sottofondo utilizzando Demucs
- Sincronizzazione avanzata con rilevamento delle pause naturali
- Analisi completa delle metriche di sincronizzazione
- Gestione migliorata della cache per i campioni vocali

## 1. Installazione

Prima di tutto, cloniamo il repository e installiamo le dipendenze necessarie:

In [ ]:
# Clona il repository
!git clone https://github.com/mikegazzaruso/video_translator.git
%cd video_translator

# Installa le dipendenze
!pip install -r requirements.txt

# Installa ffmpeg se non Ã¨ giÃ  installato
!apt-get update
!apt-get install -y ffmpeg

## 2. Carica il Video

Carica il video che desideri tradurre:

In [ ]:
%cd video_translator
from google.colab import files
import os

# Crea una directory per i file di input
!mkdir -p input_files

# Carica il file video
print("Carica il tuo video:")
uploaded = files.upload()
input_video_path = list(uploaded.keys())[0]
!mv "$input_video_path" input_files/
input_video_path = f"input_files/{input_video_path}"
print(f"Video caricato in: {input_video_path}")

## 3. Carica i Campioni Vocali (Opzionale)

Se vuoi clonare una voce specifica, carica i campioni vocali (file WAV):

In [ ]:
# Crea una directory per i campioni vocali
!mkdir -p voice_samples

# Carica i campioni vocali
print("Carica i campioni vocali (file WAV):")
uploaded_voices = files.upload()
for filename in uploaded_voices.keys():
    !mv "$filename" voice_samples/
    print(f"Campione vocale caricato: {filename}")

voice_samples_dir = "voice_samples"

## 4. Configura le Opzioni di Sincronizzazione

Imposta le opzioni di sincronizzazione per un migliore allineamento tra l'audio originale e quello tradotto:

In [ ]:
import json

# Definisci le opzioni di sincronizzazione
sync_options = {
    "max_speed_factor": 1.3,        # Fattore massimo di accelerazione
    "min_speed_factor": 0.8,        # Fattore minimo di rallentamento
    "pause_threshold": -35,         # Soglia dB per il rilevamento delle pause
    "min_pause_duration": 150,      # Durata minima della pausa in ms
    "adaptive_timing": True,        # Usa timing adattivo basato sulla lingua
    "preserve_sentence_breaks": True # Preserva le pause tra le frasi
}

# Salva le opzioni di sincronizzazione in un file
with open("sync_config.json", "w") as f:
    json.dump(sync_options, f, indent=2)

print("Opzioni di sincronizzazione salvate in sync_config.json")

## 5. Traduci il Video

Ora, traduciamo il video utilizzando lo script autodub.py. Puoi utilizzare varie opzioni di gestione della cache:

- `--no-cache`: Disabilita completamente la cache (forza la generazione di nuovi conditioning latents)
- `--clear-cache`: Cancella tutti i dati nella cache prima dell'elaborazione
- `--clear-voice-cache`: Cancella solo la cache vocale prima dell'elaborazione

Per questo esempio, useremo le impostazioni di cache predefinite:

In [ ]:
# @title
# Definisci il percorso di output
output_video_path = "output_video.mp4"

# Definisci le lingue di origine e destinazione
source_lang = "it"  # Italiano
target_lang = "en"  # Inglese

# Esegui lo script di traduzione
!python autodub.py \
    --input "$input_video_path" \
    --output "$output_video_path" \
    --source-lang "$source_lang" \
    --target-lang "$target_lang" \
    --voice-samples "$voice_samples_dir" \
    --sync-config "sync_config.json" \
    --keep-temp

## 6. Analizza le Metriche di Sincronizzazione

Analizziamo le metriche di sincronizzazione per valutare la qualitÃ  della traduzione:

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os

# Trova il file delle metriche di sincronizzazione
conversions_dir = "conversions"
latest_dir = sorted([os.path.join(conversions_dir, d) for d in os.listdir(conversions_dir) if os.path.isdir(os.path.join(conversions_dir, d))], key=os.path.getmtime)[-1]
metrics_file = os.path.join(latest_dir, "sync_metrics.json")

# Carica le metriche di sincronizzazione
with open(metrics_file, "r") as f:
    metrics = json.load(f)

# Stampa le metriche di sincronizzazione
print("Metriche di QualitÃ  della Sincronizzazione:")
print(f"  Punteggio di allineamento complessivo: {metrics['overall_alignment_score']:.2f}")
print(f"  Punteggio DTW: {metrics['dtw_score']:.2f}")
print(f"  Errore medio di timing: {metrics['avg_timing_error']:.2f} ms")
print(f"  Errore massimo di timing: {metrics['max_timing_error']:.2f} ms")
print(f"  Percentuale di segmenti ben allineati: {metrics['percent_well_aligned']:.1f}%")

# Visualizza i punteggi dei segmenti
segment_scores = [segment['sync_score'] for segment in metrics['segment_scores']]
segment_delays = [segment['delay'] for segment in metrics['segment_scores']]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

# Visualizza i punteggi di sincronizzazione
ax1.bar(range(len(segment_scores)), segment_scores, color='skyblue')
ax1.set_title('Punteggi di Sincronizzazione dei Segmenti')
ax1.set_xlabel('Indice del Segmento')
ax1.set_ylabel('Punteggio di Sincronizzazione')
ax1.axhline(y=70, color='r', linestyle='--', label='Soglia di Buona Sincronizzazione')
ax1.legend()

# Visualizza i ritardi di timing
ax2.bar(range(len(segment_delays)), segment_delays, color='lightgreen')
ax2.set_title('Ritardi di Timing dei Segmenti')
ax2.set_xlabel('Indice del Segmento')
ax2.set_ylabel('Ritardo (ms)')
ax2.axhline(y=0, color='k', linestyle='-')
ax2.axhline(y=100, color='r', linestyle='--', label='Soglia di Ritardo Accettabile')
ax2.axhline(y=-100, color='r', linestyle='--')
ax2.legend()

plt.tight_layout()
plt.show()

## 7. Visualizza e Scarica il Video Tradotto

Infine, visualizziamo e scarichiamo il video tradotto:

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Visualizza il video
mp4 = open(output_video_path, 'rb').read()
data_url = f"data:video/mp4;base64,{b64encode(mp4).decode()}"
HTML(f"""
<video width="640" height="360" controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
# Scarica il video tradotto
files.download(output_video_path)

# Trova e scarica il file di visualizzazione della sincronizzazione
visualization_file = os.path.join(latest_dir, "sync_debug.wav")
if os.path.exists(visualization_file):
    print("Scaricando il file di visualizzazione della sincronizzazione...")
    files.download(visualization_file)